In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool, Tool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


def plus(inputs):
    a, b = inputs.split(",")
    return float(a) + float(b)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    tools=[
        Tool.from_function(
            func=plus,
            name="Sum Calculator",
            description="Use this to perform sums of two numbers. Use this tool by sending a pair of number separated by a comma.\nExample:1,2",
        ),
    ],
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)



> Entering new AgentExecutor chain...
Thought: I can use the Sum Calculator tool to find the total cost of all the individual amounts provided by the user.

Action:
```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 355.39,
    "b": 924.87
  }
}
```
Observation: 1280.26
Thought:```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 1280.26,
    "b": 721.2
  }
}
```
Observation: 2001.46
Thought:```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 2001.46,
    "b": 1940.29
  }
}
```
Observation: 3941.75
Thought:```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 3941.75,
    "b": 573.63
  }
}
```
Observation: 4515.38
Thought:```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 4515.38,
    "b": 65.72
  }
}
```
Observation: 4581.1
Thought:```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 4581.1,
    "b": 35.00
  }
}
```
Observation: 4616.1
Thought:```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 

{'input': 'Tell me Total Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': 'I have calculated the total cost of all the individual amounts provided by the user. The final answer is $5273.38.'}